# Backprop From Scratch
Following this [post](https://towardsdatascience.com/backpropagation-made-easy-e90a4d5ede55).

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from numpy.random import normal
from copy import copy

## Network Construction

In [2]:
class NN:
    def __init__(self, input_dim, layer_dims):
        dims = [input_dim] + layer_dims

        # Weights
        self.W = []
        for i in range(1, len(dims)):
            # Begin with random weights
            self.W.append( normal(size=(dims[i], dims[i-1])) )

        # Biases
        self.b = [np.zeros((d, 1)) for d in layer_dims]

        # Weighted sums
        self.z = [np.zeros((d, 1)) for d in layer_dims]

        # Weighted sum activation values
        self.a = [np.zeros((d, 1)) for d in layer_dims]

In [3]:
net = NN(2, [3, 2])

## Feed-Forward

In [4]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_(x):
    return sigmoid(x) * (1 - sigmoid(x))


class NN:
    def __init__(self, input_dim, layer_dims):
        dims = [input_dim] + layer_dims

        # Weights
        self.W = []
        for i in range(1, len(dims)):
            # Begin with random weights
            self.W.append( normal(size=(dims[i], dims[i-1])) )

        # Biases
        self.b = [np.zeros((d, 1)) for d in layer_dims]

        # Weighted sums
        self.z = [np.zeros((d, 1)) for d in layer_dims]

        # Weighted sum activation values
        self.a = [np.zeros((d, 1)) for d in layer_dims]


    def ff(self, x):
        for l in range(len(self.W)):
            prev_a = (self.a[l-1] if l > 0 else x)

            # Get weighted sum of activations from previous layer into each node
            # of current layer
            self.z[l] = np.matmul(self.W[l], prev_a) + self.b[l]

            # Apply activation function to weighted sum
            self.a[l] = sigmoid(self.z[l])

        return self.a[-1]

In [5]:
net = NN(2, [3, 2])

x = np.array([[1], [4]])
net.ff(x)

array([[0.36098376],
       [0.65881161]])

## Error propagation

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_(x):
    return sigmoid(x) * (1 - sigmoid(x))


class NN:
    def __init__(self, input_dim, layer_dims):
        dims = [input_dim] + layer_dims

        # Weights
        self.W = []
        for i in range(1, len(dims)):
            # Begin with random weights
            self.W.append( normal(size=(dims[i], dims[i-1])) )

        # Biases
        self.b = [normal(size=(d, 1)) for d in layer_dims]

        # Weighted sums
        self.z = [np.zeros((d, 1)) for d in layer_dims]

        # Weighted sum activation values
        self.a = [np.zeros((d, 1)) for d in layer_dims]


    def ff(self, x):
        for l in range(len(self.W)):
            prev_a = (self.a[l-1] if l > 0 else x)

            # Get weighted sum of activations from previous layer into each node
            # of current layer
            self.z[l] = np.matmul(self.W[l], prev_a) + self.b[l]

            # Apply activation function to weighted sum
            self.a[l] = sigmoid(self.z[l])

        return self.a[-1]


    def backprop(self, x, y, alpha=0.05):
        # Calculate gradients for weights and biases
        # with respect to loss function
        dC_dz = copy(self.z)
        dC_dW = copy(self.W)
        dC_db = copy(self.b)
        for i in reversed(range(len(self.W))):
            if i == len(self.W) - 1:
                # Derivative of quadratic loss function (cost) relative
                # to the activation values (prediction)
                dC_da = (y - self.a[-1])

                # BP 1: derivative of cost relative weighted sum
                #       (dC/da[-1]) * (da[-1]/dz[-1]) = dC/dz[-1]
                dC_dz[i] = np.multiply(dC_da, sigmoid_(self.z[i])) # Element-wise multiplication
            else:
                # BP 2: derivative of cost function relative to weighted sum
                #       dC/da[i]
                #        = (dC/dz[i+1]) * (dz[i+1]/da[i]) --> this ratio is the weight!
                #        = (dC/dz[i+1]) * W[i+1] --> this gets transposed and placed on left to line everything up
                #
                #       dC/dz[i]
                #        = (dC/da[i]) * (da[i]/dz[i])
                #        = (dC/da[i]) * activation'(dz[i])
                dC_dz[i] = np.multiply(np.matmul(self.W[i+1].T, dC_dz[i+1]), sigmoid_(self.z[i]))

            # BP 3
            dC_db[i] = dC_dz[i]

            # BP 4
            if i == 0:
                dC_dW[i] = np.matmul(dC_dz[i], x.T)
            else:
                dC_dW[i] = np.matmul(dC_dz[i], self.a[i-1].T)

        # Update weights and baises
        for i in range(len(self.W)):
            self.W[i] += dC_dW[i] * alpha
            self.b[i] += dC_db[i] * alpha

In [7]:
net = NN(2, [3, 2])

x = np.array([[1], [4]])
net.ff(x)

y = np.array([[0], [1]])
net.backprop(x, y)

## Example problem

In [8]:
iris = load_iris()
X = iris.data
Y = np.zeros((iris.target.shape[0], 3))
for i, idx in enumerate(iris.target):
    Y[i, idx] = 1

def test(net, X, Y):
    correct = 0
    for i in range(X.shape[0]):
        x = X[i].reshape(X.shape[1], 1)
        pred = np.argmax(net.ff(x))
        correct += (pred == iris.target[i])

    return correct / X.shape[0]

In [9]:
net = NN(X.shape[1], [2, 3])

iterations = 1000
for i in range(1, iterations+1):
    for j in range(X.shape[0]):
        x = X[j].reshape(X.shape[1], 1)
        net.ff(x)
        net.backprop(x, Y[j].reshape(3, 1), alpha=0.01)

    if i % (iterations // 10) == 0:
        print(i, test(net, X, Y))

100 0.6666666666666666
200 0.7333333333333333
300 0.7533333333333333
400 0.7866666666666666
500 0.7933333333333333
600 0.8
700 0.8
800 0.8
900 0.8
1000 0.8066666666666666
